In [ ]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel

plt.rcParams['figure.figsize'] = (11, 8)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['text.usetex'] = True

import sys

sys.path.insert(0, '..')
from modules import structure_func as sf
from modules import regression as rg

In [ ]:
data = pd.read_table('../data/cosmograil_data/RXJ1131_Tewes2013.rdb_-1.txt')

t = data['mhjd'].to_numpy(dtype=np.float64)
A = data['mag_A'].to_numpy(dtype=np.float64)
B = data['mag_B'].to_numpy(dtype=np.float64)
errA = data['magerr_A'].to_numpy(dtype=np.float64)
errB = data['magerr_B'].to_numpy(dtype=np.float64)

N = len(t)

In [ ]:
plt.scatter(t, A, label='A')
plt.legend()

In [ ]:
diffs = np.diff(t)
mean_diff = diffs.mean()
small_diffs = diffs[diffs <= mean_diff]

In [ ]:
plt.hist(small_diffs, bins=50)

In [ ]:
def get_large_gaps(diffs, nsigma=3):
    mean_dt = diffs.mean()
    sigma_dt = diffs.std()
    return diffs[diffs >= mean_dt + nsigma*sigma_dt]

In [ ]:
input_files = list(Path('../data/cosmograil_data/').glob('*'))

gaps_df_dict = {}
large_gaps_dict = {}

for file in input_files:
    gaps_df = pd.DataFrame()
    qso_id = file.name.split('_')[0]
    for nsigma in [1,2,3]:
        qso_df = pd.read_table(file, delim_whitespace=True)
        t = qso_df['mhjd'].values
        diffs = np.diff(t)
        large_gaps = get_large_gaps(diffs, nsigma=nsigma)
        data_dict = {'gapsize_mean': large_gaps.mean(), 
                     'gapsize_sigma': large_gaps.std(),
                     'n_gaps': len(large_gaps), 
                     'obs_time': t[-1]-t[0], 
                     'mean_sampling_rate': 1./diffs.mean(),
                     'n_samples': len(t)}
        gaps_df = gaps_df.append(pd.DataFrame(data_dict, index=[nsigma]))
    large_gaps_dict[qso_id] = get_large_gaps(diffs, nsigma=3)
    gaps_df_dict[qso_id] = gaps_df

df = pd.concat(gaps_df_dict, names=['qso', 'nsigma'])

df

In [ ]:
diffs = np.diff(t)
mean_diff = diffs.mean()
sigma_diff = diffs.std()
season_ends = t[np.where(diffs >= mean_diff + 3*sigma_diff)[0]] 
seasons = []

for i in range(len(season_ends) - 1):
    start = season_ends[i]
    end = season_ends[i+1]
    seasons.append(t[(t > start) & (t <= end)])


In [ ]:
for s in seasons:
    plt.scatter(s, 0*s)

In [ ]:
plt.scatter(seasons[0], seasons[0]*0)

In [ ]:
T = 1000
step = 5
t_base = np.arange(0, T, step)
np.random.seed(1234)
noise = np.random.normal(t_base, step/3, size=len(t_base))
#noise += np.random.normal(t_base+step/4, step/3, size=len(t_base))
myt = t_base + noise

In [ ]:
plt.hist(np.diff(myt))